In [60]:
import tensorflow as tf 
import librosa
import pandas as pd
import torch.optim as optim
import numpy as np 
import csv 
import os
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import librosa
from librosa.core import stft, magphase
from glob import glob
from torch import autograd
import csv
from data_load import CodeSwitchDataset
import zipfile

In [2]:
train_dataset = CodeSwitchDataset(lang = 'Telugu', mode = "train")

In [3]:
train_loader = DataLoader(train_dataset,
                          batch_size=32,
                          shuffle=True,
                          drop_last=True)

In [4]:
device = torch.device('cuda')

In [5]:
#for batch_idx, (wav, label) in enumerate(train_loader):
#    print(wav.shape)

In [6]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim = 2, num_layers = 4):
        super(Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        #self.drop = nn.Dropout(0.25)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, x):
        #print(type(self.lstm))
        print(x.shape)
        lstm_out, hidden = self.lstm(x)
        logits = self.linear(lstm_out[-1])
        class_scores = F.log_softmax(logits, dim=1)
        return class_scores

In [7]:
model = Model(input_dim=1025,
              hidden_dim=512,
              batch_size=8,
              output_dim=2,
              num_layers=4).to(device)
criterion = nn.CTCLoss(reduction='none').to(device)
epochs = 40
optimizer = optim.Adam(model.parameters())

In [8]:
device

device(type='cuda')

In [9]:
model.half()

Model(
  (lstm): LSTM(1025, 512, num_layers=4)
  (linear): Linear(in_features=512, out_features=2, bias=True)
)

In [9]:
for layer in model.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
for e in range(epochs):
    training_loss, train_acc = 0, 0
    for batch_idx, (wav, label) in enumerate(train_loader):
        #bi, wav, label = batch_idx, wav, label
        wav = wav.to(device).half()
        label = label.to(device)
        model.zero_grad()
    
        out = model(wav)
        out = out.transpose(0,1)
        
        criterion()
        print(wav.shape)

torch.Size([32, 2409, 1025])


RuntimeError: cuDNN error: CUDNN_STATUS_BAD_PARAM

In [ ]:
y_pred.shape

In [ ]:
y_pred

In [21]:
wav, sr = librosa.load('Data/PartA_Telugu/Train/Audio/000010010.wav')
df = pd.read_csv('Data/PartA_Gujarati/Train/Transcription_LT_Sequence.tsv', header=None, sep='\t')

In [36]:
def pad(wav, max_len):
    while len(wav) < max_len:
        diff = max_len - len(wav)
        ext = wav[:diff]
        wav = np.append(wav, wav[:diff])
    return wav

def preprocess(wav, sr):
    out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
    out = magphase(out)[0]
    out = [np.log(1 + x) for x in out]
    return np.array(out)
wav = pad(wav,529862)
out = preprocess(wav, sr)
out = np.transpose(out, axes=(1, 0))

In [64]:
out.shape

torch.Size([2409, 2, 1025])

In [41]:
out = np.dstack((out, out))

In [44]:
out = np.transpose(out, axes=(0,2,1))

In [ ]:
T, B, C = 2409, 32, 1025
input_lengths = torch.full((B,), T, dtype = torch.long, device = device)

In [ ]:
input_lengths.shape

In [ ]:
t = T // 2 - 4

In [ ]:
target_lengths = torch.full((B,), t, dtype = torch.long, device = device)

In [ ]:
target_lengths

In [61]:
out = torch.tensor(out)
log_probs = model(out)
input_lengths=torch.full((model(out).shape[1],), log_probs.shape[0],   dtype=torch.long);
input_lengths=autograd.Variable(input_lengths)
target_lengths=autograd.Variable(target_lengths)

/home/dexter/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([2409, 2, 1025])
torch.Size([2409, 2, 1025])


NameError: name 'target_lengths' is not defined

In [62]:
input_lengths

tensor([2, 2])

In [63]:
log_probs

tensor([[-0.6965, -0.6898],
        [-0.6965, -0.6898]], grad_fn=<LogSoftmaxBackward>)

In [13]:
torch.cuda.empty_cache()

In [56]:
next(model.parameters()).is_cuda

False

In [55]:
model.cpu()

Model(
  (lstm): LSTM(1025, 512, num_layers=4)
  (linear): Linear(in_features=512, out_features=2, bias=True)
)